# Import packages

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns

ImportError: cannot import name 'Image' from 'PIL' (unknown location)

# Import the dataframe

In [ ]:
# Import the dataframe
df: pd.DataFrame = pd.read_csv('Datasets/cleaned_data.csv')

# Change the Dtype of all the date to DateType instead of an object
df['datum'] = pd.to_datetime(df['datum'])
df['geboortedatum'] = pd.to_datetime(df['geboortedatum'], dayfirst=True)
df['startdatum'] = pd.to_datetime(df['startdatum'])
df['einddatum'] = pd.to_datetime(df['einddatum'])

# Show all the column names
df.info()

# Feature engineering

# Create new dummies

In [ ]:
df: pd.DataFrame = pd.get_dummies(df, columns=['afdeling'])
df = pd.get_dummies(df, columns=['business_unit'])
df = pd.get_dummies(df, columns=['uitdienst_reden'])
dummy_row_gender = pd.get_dummies(df['geslacht'], prefix='geslacht_id')
df = pd.concat([df, dummy_row_gender], axis=1)
df.columns

In [ ]:
# Visualisation with dummies

In [ ]:
#Visualisaties met dummies
#Histogram van het aantal werknemers met STATUS beëindiging per jaar

df_beindigd = df[df['status_beëindigd'] == 1]

plt.figure(figsize=(10, 6))
plt.bar(df_beindigd['status_jaar'].value_counts().index, df_beindigd['status_jaar'].value_counts().values, color='blue', edgecolor='black', label='Aantal werknemers met STATUS_beëindiging')
plt.axhline(y=df_beindigd['status_jaar'].value_counts().mean(), color='red', linestyle='dashed', linewidth=2, label='Gemiddelde')

plt.xlabel('Jaar')
plt.ylabel('Aantal werknemers met status beëindiging')
plt.title('Aantal werknemers met status beëindiging per jaar')
plt.legend()
plt.show()

In [ ]:
#deze plot is gemaakt om te zien of de leeftijd erg verschilt per stad, dit blijkt mee te vallen
#hierdoor is er gekozen om geen andere technieken te gebruiken dan de modus om NaN values in stad in te vullen met Overige
import pandas as pd
import matplotlib.pyplot as plt

average_age_per_city = df.groupby('stad')['leeftijd'].mean().reset_index()

plt.figure(figsize=(10, 6))
plt.bar(average_age_per_city['stad'], average_age_per_city['leeftijd'], color='blue', edgecolor='black')
plt.xlabel('City')
plt.ylabel('Average Age')
plt.title('Gemiddelde leeftijd per stad')
plt.xticks(rotation=45, ha='right')  
plt.tight_layout()

plt.show()

In [ ]:
#om te onderzoeken in welke steden er veel uitstroom van werknemers is maken we deze plot
reasons_df = df[['stad', 'uitdienst_reden_Ontslag', 'uitdienst_reden_Vertrek']]

reasons_df = reasons_df.fillna(0)

reasons_count_per_city = reasons_df.groupby('stad').sum().reset_index()

melted_reasons = pd.melt(reasons_count_per_city, id_vars='stad', var_name='Reason', value_name='Count')

plt.figure(figsize=(12, 8))
sns.barplot(x='stad', y='Count', hue='Reason', data=melted_reasons, color='blue', edgecolor='black')
plt.xlabel('City')
plt.ylabel('Count')
plt.title('Aantal werknemers per reden voor weggaan per stad')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.show()

In [ ]:
reasons_df = df[['stad', 'uitdienst_reden_Ontslag', 'uitdienst_reden_Vertrek']]

reasons_df = reasons_df.fillna(0)

reasons_count_per_city = reasons_df.groupby('stad').sum().reset_index()

total_count_per_city = df['stad'].value_counts().reset_index()
total_count_per_city.columns = ['stad', 'total_count']

merged_df = pd.merge(reasons_count_per_city, total_count_per_city, on='stad')

merged_df['percentage_Ontslag'] = (merged_df['uitdienst_reden_Ontslag'] / merged_df['total_count']) * 100
merged_df['percentage_Vertrek'] = (merged_df['uitdienst_reden_Vertrek'] / merged_df['total_count']) * 100

melted_reasons = pd.melt(merged_df, id_vars='stad', var_name='Reason', value_name='Percentage')

melted_reasons = melted_reasons[melted_reasons['Reason'].isin(['percentage_Ontslag', 'percentage_Vertrek'])]

plt.figure(figsize=(12, 8))
sns.barplot(x='stad', y='Percentage', hue='Reason', data=melted_reasons)
plt.xlabel('City')
plt.ylabel('Percentage of Total Count')
plt.title('Percentage of Reasons for Leaving per City')
plt.xticks(rotation=45, ha='right') 
plt.tight_layout()

plt.show()

In [ ]:
#om te bekijken wat de leeftijdsverdeling van uitgestroomde werknemers is per piek in de vertrekken per stad
#is deze plot gemaakt
cities_to_plot = ['Alblasserdam', 'Barendrecht', 'Middelburg', 'Veenendaal', 'Woerden']

for city in cities_to_plot:
    filtered_df = df[(df['uitdienst_reden_Ontslag'] == 1) & (df['stad'] == city)]

    plt.figure(figsize=(10, 6))
    plt.hist(filtered_df['leeftijd'], bins=20, color='blue', edgecolor='black')
    plt.xlabel('Age')
    plt.ylabel('Count')
    plt.title(f'Count of Ages with Ontslag=1 in {city}')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    plt.show()

In [ ]:
#percentages mannen en vrouwen, vertrokken en ontslagen
# leaving_age_df = df[df['uitdienst_reden'] != 'Niet van toepassing']

percentage_ontslagen_mannen = (df[df['geslacht_id_Man'] == 1]['uitdienst_reden_Ontslag'].sum() / len(df[df['geslacht_id_Man'] == 1])) * 100
percentage_vertrokken_mannen = (df[df['geslacht_id_Man'] == 1]['uitdienst_reden_Vertrek'].sum() / len(df[df['geslacht_id_Man'] == 1])) * 100

percentage_ontslagen_vrouwen = (df[df['geslacht_id_Vrouw'] == 1]['uitdienst_reden_Ontslag'].sum() / len(df[df['geslacht_id_Vrouw'] == 1])) * 100
percentage_vertrokken_vrouwen = (df[df['geslacht_id_Vrouw'] == 1]['uitdienst_reden_Vertrek'].sum() / len(df[df['geslacht_id_Vrouw'] == 1])) * 100

plt.figure(figsize=(10, 5))
labels = ['Ontslagen Vrouwen', 'Vertrokken Vrouwen']
sizes = [percentage_ontslagen_vrouwen, percentage_vertrokken_vrouwen]
colors = ['red', 'pink']

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Percentage ontslagen en vertrokken vrouwen')
plt.show()

plt.figure(figsize=(10, 5))
labels = ['Ontslagen Mannen', 'Vertrokken Mannen']
sizes = [percentage_ontslagen_mannen, percentage_vertrokken_mannen]
colors = ['blue', 'cyan']

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Percentage ontslagen en vertrokken mannen')
plt.show()

In [ ]:
cities_to_plot = ['Alblasserdam', 'Barendrecht', 'Middelburg', 'Veenendaal', 'Woerden']

for city in cities_to_plot:
    df_city = df[df['stad'] == city]

    count_ontslag_1 = df_city[df_city['uitdienst_reden_Ontslag'] == 1]['leeftijd']
    count_ontslag_0 = df_city[df_city['uitdienst_reden_Ontslag'] == 0]['leeftijd']

    plt.figure(figsize=(12, 6))

    plt.hist([count_ontslag_1, count_ontslag_0], bins=20, stacked=True, color=['blue', 'orange'], edgecolor='black', label=['Ontslag=1', 'Ontslag=0'])
    plt.xlabel('Age')
    plt.ylabel('Count')
    plt.title(f'Count of Ages in {city}')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

In [ ]:
#column aantal geboortes aangemaakt
birth_df = pd.read_csv('Datasets/geboortecijfers_2022.csv', sep=';', decimal=',')
birth_df
birth_df.replace('Pijnacker-Nootdorp', 'Pijnacker', inplace=True)
birth_df.replace('s-Gravenhage', 'Den Haag', inplace=True)
birth_df.replace('Zaanstad', 'Zaandam', inplace=True)
birth_df.replace('Capelle aan den IJssel', 'Capelle', inplace=True)
# Assuming birth_df and start_df are your DataFrames
merged_df = df.merge(birth_df, left_on='stad', right_on='Gemeente', how='left')
birth_df.info()
# Add the required columns to start_df
df['aantal_geboortes'] = merged_df['Levend geborenen per duizend inwoners']
df['aantal_geboortes_pf'] = merged_df["Levend geborenen per duizend vrouwen (15 tot 50 jaar)"]

In [ ]:
df.info()

In [ ]:
df = df[df['status_beëindigd'] == True]
df['leeftijd_begin_dienst'] = df['leeftijd'] - df['dienstperiode']


# Store the Dataframe 

In [ ]:
df.to_csv('Datasets/trainingdata.csv', sep=',', index=False)